In [1]:
import cv2
import os

import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

In [2]:
dirs = ["../data/" + k + "/" for k in os.listdir("../data/")]
dirs[:5]

['../data/anger/',
 '../data/contempt/',
 '../data/disgust/',
 '../data/fear/',
 '../data/happy/']

In [3]:
IMAGE_FILES = []

for d in dirs:
    IMAGE_FILES += [d+k for k in os.listdir(d)]

IMAGE_FILES[:3], len(IMAGE_FILES)

(['../data/anger/S010_004_00000017.png',
  '../data/anger/S010_004_00000018.png',
  '../data/anger/S010_004_00000019.png'],
 981)

In [4]:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

In [5]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

point = Point(0,0)
polygon = Polygon([(11,16),(15,15),(18,18)])
print(polygon.contains(point))

False


In [6]:
from skimage import feature
import numpy as np

class LocalBinaryPatterns:
    
    def __init__(self, numPoints, radius):
        # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius
        
    def describe(self, image, eps=1e-7):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        lbp = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, self.numPoints + 3), range=(0, self.numPoints + 2))
        
        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        
        # return the histogram of Local Binary Patterns
        return hist

In [7]:
desc = LocalBinaryPatterns(16, 8)

In [8]:
lbp_feat = {}

In [9]:
count = 0

for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    if not results.multi_face_landmarks:
        continue
        
    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
        
        poly_indices = [
            # face contour
#             [10,  338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288,
#             397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136,
#             172, 58,  132, 93,  234, 127, 162, 21,  54,  103, 67,  109,],
            
            # lips
            [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291,375,321,405,314,17,84,181,91,146],
            
            # nose
            [1,2,98,168],

            #midwayBetweenEyes
            # 168,
            
            #rightEye
            [247, 30, 29, 27, 28, 56, 190,243,112,26,22,23,24,110,25,130],
#             [29, 28,243,22,24,130,29],
            
            #rightEyebrow
            [247, 30, 29, 27, 28, 56, 190,193,55,107,66,105,63,70,156],
            
            [291,375,321,405,314,17,84,181,176,148,152,377,400,378,379,365,397],
            
            [327,454,323,291],
            
            [130,50,93,234],
            
        ]
        
        lbp_feat[file] = np.array([])
        
        c = 0
        for p in poly_indices:
            annotated_image = image.copy()
            
            allowedPoints = []
            polygon = []
            for i in p:
                k = face_landmarks.landmark[i]
                x,y,z = k.x,k.y,k.z
                x = int(annotated_image.shape[0]*x)
                y = int(annotated_image.shape[1]*y)
                # annotated_image = cv2.circle(annotated_image, (x,y), radius=0, color=(0, 0, 255), thickness=-1)
                polygon.append((x,y))
                allowedPoints.append([x,y])
                
            polygon = Polygon(polygon)
                
            for i in range(annotated_image.shape[0]):
                for j in range(annotated_image.shape[1]):
                    if polygon.contains(Point(i,j)):
                        allowedPoints.append([i,j])
                        
            for i in range(annotated_image.shape[0]):
                for j in range(annotated_image.shape[1]):
                    if [i,j] not in allowedPoints:
                        annotated_image[j][i] = 0
#             c += 1
#             cv2.imwrite("../data/anger/" + str(c) + '_1.png', annotated_image)
                    
            gray = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2GRAY)
            hist = desc.describe(gray)

            lbp_feat[file] = np.append(lbp_feat[file],hist,axis=0)
        
    count += 1
    if count%10 == 0:
        print(count)
        
#         cv2.imwrite("../data/anger/" + str(idx) + '_removed.png', annotated_image)
        
#     break

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980


In [10]:
len(lbp_feat)

981

In [11]:
import h5py

f = h5py.File("lbp_feat_v2.h5", "w")

for k in lbp_feat:
    f[k.replace("/","__")] = lbp_feat[k]

f.close()

In [12]:
s = []

for k in lbp_feat:
    s.append(lbp_feat[k].shape)
    
list(set(s))

[(126,)]

In [14]:
lbp = {}

with h5py.File("lbp_feat_v2.h5", "r") as f:
    for k in list(f.keys()):
        lbp[k] = np.array(f[k])
        
len(lbp)

981